# Steam - Web Scraping

In [174]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from pandas import json_normalize
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By  
from IPython.display import clear_output

In [175]:
# Selenium setup
options=Options()
options.binary_location=r"C:\Users\nclee\AppData\Local\Chromium\Application\chrome.exe"

In [176]:
# 300 Top Seller for 4 genres
def scrape_app_id():
    list_app_id=[]
    genres=["Action"]
    #genres=["Action", "Adventure", "RPG", "Simulation"]
    for genre in genres:
        for i in range(1):
            url=f"https://store.steampowered.com/contenthub/querypaginated/tags/TopSellers/render/?query=&start={i*15}&count=15&cc=FR&l=english&v=4&tag={genre}"
            results=requests.get(url).json()
            list_app_id.extend([link.strip('/')[4:] for link in re.findall(r'/app/.*?/',results['results_html'],re.MULTILINE)])
            if (i+1)%5==0:
                print(f"Successfully scraped #{i+1} pages of app id in {genre} genre.")
                time.sleep(2)
    print(f"Scraping of app id completed.")
    return set(list_app_id)

In [187]:
# Grabbing tags
def grab_tags(app_id):
    tag_link=f"https://store.steampowered.com/apphoverpublic/{app_id}?review_score_preference=0&l=english&pagev6=true"
    soup = BeautifulSoup(requests.get(tag_link).content)
    tag = ",".join([i.text for i in soup.select("div.app_tag")])
    try:
        rating = soup.select("div.hover_review_summary span")[0].text
    except:
        rating=""
    try:
        nreview = int(re.findall(r"\(.*?\)", soup.select('div.hover_review_summary')[0].text)[0].strip(f"( reviews)").replace(",", ""))
    except:
        nreview=""
    return tag, rating, nreview

In [199]:
app_id=1174180

In [200]:
# review
def grab_reviews(app_id):
    review_link=f"https://store.steampowered.com/appreviews/{app_id}?cursor=*&day_range=30&start_date=-1&end_date=-1&date_range_type=all&filter=summary&language=english&l=english"
    
    results=requests.get(review_link).json()
    results=results['html']
    soup = BeautifulSoup(results)
    
    review=[(i.text.strip("\r\n\t").replace("<br/>", "\n")) for i in soup.select("div.content")]
    
    return review

In [191]:
def grab_html(app_id):
    url=f"https://store.steampowered.com/agecheck/app/{app_id}"
    browser= webdriver.Chrome(options=options)
    browser.get(url)
    delay = 3
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'game_highlights')))
    except:
        try:
            input_day = browser.find_element_by_css_selector("select#ageDay")
            input_month = browser.find_element_by_css_selector("select#ageMonth")
            input_year = browser.find_element_by_css_selector("select#ageYear")
            input_day.send_keys("23")
            input_month.send_keys("August")
            input_year.send_keys("1986")
        except:
            pass
        finally:
            button_ele = browser.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_medium")
            button_ele.click()
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'game_highlights')))
    else:
        pass
    finally:
        elem=browser.find_element_by_xpath('//*')
        text=elem.get_attribute('innerHTML')
        soup=BeautifulSoup(text)
        browser.quit()
        return soup


In [193]:
def extract_info(soup):

    title=soup.select("div.apphub_AppName")[0].text
    try:
        release_date=soup.select("div.release_date div.date")[0].text
    except:
        release_date=""
    try:
        publisher=", ".join([i.text.strip("\n ") for i in soup.select("div.dev_row+div.dev_row>div.summary.column")])
    except:
        publisher=""
    try:
        summary=soup.select("div.game_description_snippet")[0].text.strip("\r\t\n")
    except:
        summary=""
        
    try:
        platform=", ".join([i.get("class")[1] for i in set(soup.select("div.game_area_purchase_platform span"))])
    except:
        platform=""
    try:
        language= ", ".join([row.text.strip("\n\r\t").split("\t")[0] for row in soup.select("table.game_language_options tr")[1:]])
    except:
        language=""
    try:
        PEGI=soup.select("div.game_rating_icon a img")[0].get('src').split("/")[-1].strip(".png")
    except:
        PEGI=""

    genre=", ".join([i.text for i in soup.select("div.block_content_inner div.details_block b~a") if i.get("href").startswith("https://store.steampowered.com/genre/")])

    try:
        site=soup.select("div.details_block+div.details_block a")[0].get("href").split("?url=")[1]
    except:
        site=""

    try:
        metacritic=int([i.strip("\t\n\r") for i in re.findall(r'\d*?\t',soup.select("div#game_area_metascore")[0].text,re.MULTILINE) if len(i.strip("\t\n\r"))>0][0])
    except:
        metacritic=""
    try:
        minimum="\n".join([i.text for i in soup.select("div.game_area_sys_req_leftCol ul.bb_ul li")])
    except:
        minimum=""
    try:
        recommended="\n".join([i.text for i in soup.select("div.game_area_sys_req_rightCol ul.bb_ul li")])
    except:
        recommended=""
    try:
        franchise=soup.select("div.details_block div.dev_row+div.dev_row+div.dev_row a")[0].text
    except:
        franchise=""

    try:
        discount_perc=soup.select("div.game_purchase_action div.game_purchase_action_bg div.discount_pct")[0].text.strip("\r\n\t%") #percent
        original_price=float(soup.select("div.game_purchase_action div.game_purchase_action_bg div.discount_prices div.discount_original_price")[0].text.strip("\r\n\t€").replace(",", ".")) #original
        final_price=float(soup.select("div.game_purchase_action div.game_purchase_action_bg div.discount_prices div.discount_final_price")[0].text.strip("\r\n\t€").replace(",", ".")) #final
    except:
        discount_perc=0
        try:
            original_price=float(soup.select("div.game_purchase_action div.game_purchase_action_bg div.game_purchase_price.price")[0].text.strip("\r\n\t€").replace(",","."))
        except:
            original_price=0
        final_price=original_price
    
    return title, release_date, publisher, genre, summary, PEGI, metacritic, platform,language, minimum, recommended, franchise, final_price, original_price, discount_perc
    

In [181]:
def acquire():
    data=[]
    # Selenium setup
    options=Options()
    options.binary_location=r"C:\Users\nclee\AppData\Local\Chromium\Application\chrome.exe"
    list_app_id=scrape_app_id()
    n_app_id=len(list_app_id)
    count=0
    clear_output(wait=True)
    for app_id in list_app_id:
        soup=grab_html(app_id)
        title, release_date, publisher, genre, summary, PEGI, metacritic, platform,language, minimum, recommended, franchise, final_price, original_price, discount_perc=extract_info(soup)
        tag, rating, nreview=grab_tags(app_id)
        review=grab_reviews(app_id)
        
        #append to data
        datum=[app_id, title, franchise, publisher, release_date, genre, summary, tag, PEGI, platform, final_price,\
               original_price, discount_perc, metacritic, rating, nreview, review, language, minimum, recommended]
        data.append(datum)
        count+=1
        if count%5==0:
            print(f"Downloaded {count} of {n_app_id} game information.")
            time.sleep(2)
    clear_output()
    print(f"Finished scraping game information.")
    return data   

TEST

In [194]:
data=acquire()

Finished scraping game information.


In [202]:
cols=["app_id", "name", "franchise", "publisher", "release_date", "genre", "summary", "user_defined_tag", "PEGI", "platform", "final_price",\
               "original_price", "discount_percentage", "metacritic_rating", "overall_rating", "number_of_reviews", "customer_reviews", "languages", "minimum_system_requirement", "recommended_system"]

In [203]:
df=pd.DataFrame(data, columns=cols)


In [205]:
df.set_index(["app_id"])
df.tail()

,app_id,name,franchise,publisher,release_date,genre,summary,user_defined_tag,PEGI,platform,...,original_price,discount_percentage,metacritic_rating,overall_rating,number_of_reviews,recommendation,customer_reviews,languages,minimum_system_requirement,recommended_system
10,782330,DOOM Eternal,DOOM,Bethesda Softworks,"20 Mar, 2020",Action,Hell’s armies have invaded Earth. Become the S...,"Action,Masterpiece,FPS,Great Soundtrack,Gore,D...",18,win,...,59.99,0.50,88,Very Positive,56920,0,"[Windows: click ""Start""Type ""powershell""Right ...","English, French, Italian, German, Spanish - Sp...",Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...
11,617290,Remnant: From the Ashes,,Perfect World Entertainment,"20 Aug, 2019","Action, Adventure, RPG",The world has been thrown into chaos by an anc...,"Souls-like,Action,RPG,Adventure,Co-op,Third-Pe...",16,win,...,39.99,0.40,78,Very Positive,22493,0,"[Great souls-like shooter, I highly recommend....","English, French, Italian, German, Spanish - Sp...",OS: Windows 7 / 8 / 10 64 bit\nProcessor: Inte...,OS: Windows 7 / 8 / 10 64 bit\nProcessor: Inte...
12,397540,Borderlands 3,Borderlands,2K,"13 Mar, 2020","Action, RPG","The original shooter-looter returns, packing b...","RPG,Action,Online Co-Op,Looter Shooter,Shooter...",18,win,...,59.99,0.50,81,Very Positive,31628,0,[ The biggest mixed bag I've ever had in ever....,"English, French, Italian, German, Spanish - Sp...",Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...
13,905370,Conqueror's Blade,,MY.GAMES,"30 May, 2019","Action, Free to Play, Massively Multiplayer, S...",Master the art of medieval warcraft in this fr...,"Action,Massively Multiplayer,Strategy,Free to ...",,win,...,0.00,0.00,,Mostly Positive,9809,0,"[This game was great at one time, but now I ca...","English, French, German, Spanish - Spain",Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...
14,1118010,Monster Hunter World: Iceborne,Monster Hunter,"CAPCOM Co., Ltd.","9 Jan, 2020",Action,,"Action,Multiplayer,Co-op,Hunting,Open World,Di...",16,win,...,39.99,0.25,,Mixed,11114,0,[I have never written a review on a game becau...,"English, French, Italian, German, Spanish - Sp...","OS: Windows® 7, 8, 8.1, 10 (64-bit required)\n...","OS: Windows® 7, 8, 8.1, 10 (64-bit required)\n..."


In [135]:
df.to_csv("ACTION.csv")

In [207]:
test=df[df["platform"].str.contains("win")]

In [210]:
test2=test[test.metacritic_rating!=""]

In [214]:
test2.sort_values(by="metacritic_rating", ascending=False, inplace=True)

<ipython-input-214-51994afc7f31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2.sort_values(by="metacritic_rating", ascending=False, inplace=True)


In [218]:
test2

,app_id,name,franchise,publisher,release_date,genre,summary,user_defined_tag,PEGI,platform,...,original_price,discount_percentage,metacritic_rating,overall_rating,number_of_reviews,recommendation,customer_reviews,languages,minimum_system_requirement,recommended_system
4,271590,Grand Theft Auto V,Grand Theft Auto,Rockstar Games,"14 Apr, 2015","Action, Adventure",Grand Theft Auto V for PC offers players the o...,"Open World,Action,Multiplayer,Automobile Sim,T...",18,win,...,29.99,0.50,96,Very Positive,790718,0,"[10/10 Post-man simulator. Im not joking, 90% ...","English, French, Italian, German, Spanish - Sp...",Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...
5,1174180,Red Dead Redemption 2,Red Dead Redemption,Rockstar Games,"5 Dec, 2019","Action, Adventure",Winner of over 175 Game of the Year Awards and...,"Adventure,Action,Open World,Masterpiece,Wester...",18,win,...,59.99,0.20,93,Mostly Positive,80701,0,[Obviously this game has been given countless ...,"English, French, Italian, German, Spanish - Sp...",Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...
1,582010,MONSTER HUNTER: WORLD,Monster Hunter,"CAPCOM Co., Ltd.","9 Aug, 2018",Action,Welcome to a new world! In Monster Hunter: Wor...,"Co-op,Multiplayer,Action,Open World,RPG,Third ...",16,win,...,29.99,0.34,88,Very Positive,151143,0,[I waited until I had at least 1000 hours on P...,"English, French, Italian, German, Spanish - Sp...",Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...
6,814380,Sekiro™: Shadows Die Twice,,"Activision (Excluding Japan and Asia), FromSof...","21 Mar, 2019","Action, Adventure","Game of the Year - The Game Awards 2019, Best ...","Souls-like,Difficult,Action,Singleplayer,Ninja...",18,win,...,59.99,0.35,88,Very Positive,62794,0,[I first played this game on the PS4 in 2019 a...,"English, French, Italian, German, Spanish - Sp...",OS: Windows 7 64-bit | Windows 8 64-bit | Wind...,OS: Windows 7 64-bit | Windows 8 64-bit | Wind...
10,782330,DOOM Eternal,DOOM,Bethesda Softworks,"20 Mar, 2020",Action,Hell’s armies have invaded Earth. Become the S...,"Action,Masterpiece,FPS,Great Soundtrack,Gore,D...",18,win,...,59.99,0.50,88,Very Positive,56920,0,"[Windows: click ""Start""Type ""powershell""Right ...","English, French, Italian, German, Spanish - Sp...",Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...


In [227]:
test[["PEGI", "final_price", "original_price", "discount_percentage"]].groupby("PEGI").mean()

,final_price,original_price,discount_percentage
PEGI,,,
,9.995000,12.495000,0.100000
12,31.923333,53.323333,0.356667
16,24.590000,36.656667,0.330000
18,28.760000,62.842857,0.505714


In [228]:
test.genre.str.get_dummies(sep=', ').sum()

Action                   15
Adventure                 7
Free to Play              1
Indie                     1
Massively Multiplayer     1
RPG                       4
Simulation                1
Strategy                  1
dtype: int64